In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os, sys

In [3]:
import pandas as pd
from loguru import logger

In [4]:
sys.path.append('..')

from pyMultiOmics.constants import *
from pyMultiOmics.mapping import Mapper
from pyMultiOmics.common import set_log_level_info
from pyMultiOmics.analysis import *
from pyMultiOmics.query import *
from pyMultiOmics.pipelines import *

2021-08-16 14:52:58.821 | WARNING  | pyMultiOmics.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2021-08-16 14:52:58.824 | WARNING  | pyMultiOmics.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2021-08-16 14:53:00.941 | INFO     | pyMultiOmics.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Demonstration of pyMultiOmics

## Load the processed Zebrafish data from [1]

[1] [Rabinowitz, Jeremy S., et al. "Transcriptomic, proteomic, and metabolomic landscape of positional memory in the caudal fin of zebrafish." Proceedings of the National Academy of Sciences 114.5 (2017): E717-E726.](https://www.pnas.org/content/114/5/E717.short)

In [5]:
DATA_FOLDER = os.path.abspath(os.path.join('test_data', 'zebrafish_data'))
DATA_FOLDER

'C:\\Users\\joewa\\Work\\git\\pyMultiOmics\\notebooks\\test_data\\zebrafish_data'

Read transcriptomics data (identified by their gene ids)

In [6]:
gene_data = pd.read_csv(os.path.join(DATA_FOLDER, 'gene_data_combined.csv'), index_col='Identifier')
gene_design = pd.read_csv(os.path.join(DATA_FOLDER, 'gene_design.csv'), index_col='sample')

Read proteomics data

In [7]:
protein_data = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_data.csv'), index_col='Uniprot')
protein_design = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_design.csv'), index_col='sample')

Read metabolomics data

In [8]:
compound_data = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_data_chebi.csv'), index_col='Identifier')
compound_design = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_design.csv'), index_col='sample')

In [9]:
set_log_level_info()

1

## Create a mapping object

The mapping object uses Reactome to map the different biological entities in the data:
- Transcripts (or genes) are connected to the proteins they encode
- Proteins and compounds are connected to reactions they're involved in
- Reactions are connected to pathways

In [10]:
m = Mapper(DANIO_RERIO, metabolic_pathway_only=True) \
        .set_gene(gene_data, gene_design) \
        .set_protein(protein_data, protein_design) \
        .set_compound(compound_data, compound_design) \
        .build()

2021-08-16 14:53:02.711 | INFO     | pyMultiOmics.functions:remove_dupes:385 - Removing 0 rows with duplicate identifiers
2021-08-16 14:53:02.712 | INFO     | pyMultiOmics.functions:reactome_mapping:78 - There are 124 observed compound ids
2021-08-16 14:53:02.712 | INFO     | pyMultiOmics.functions:reactome_mapping:81 - Mapping genes -> proteins
2021-08-16 14:53:03.436 | INFO     | pyMultiOmics.functions:reactome_mapping:86 - Mapping proteins -> reactions
2021-08-16 14:53:05.116 | INFO     | pyMultiOmics.functions:reactome_mapping:94 - Mapping compounds -> reactions
2021-08-16 14:53:05.976 | INFO     | pyMultiOmics.functions:reactome_mapping:100 - Mapping reactions -> pathways
2021-08-16 14:53:06.284 | INFO     | pyMultiOmics.functions:reactome_mapping:111 - Mapping reactions -> proteins
2021-08-16 14:53:08.687 | INFO     | pyMultiOmics.functions:reactome_mapping:118 - Mapping reactions -> compounds
2021-08-16 14:53:09.931 | INFO     | pyMultiOmics.functions:reactome_mapping:130 - Mapp

## DE analysis

In [11]:
ap = AnalysisPipeline(m)

In [12]:
method = INFERENCE_T_TEST
ap.run_de(method, GENES, 'Distal', 'Proximal')
ap.run_de(method, GENES, 'Distal', 'Middle')
ap.run_de(method, GENES, 'Proximal', 'Middle')

2021-08-16 14:53:36.765 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Distal, control is Proximal
2021-08-16 14:53:50.901 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Distal, control is Middle
2021-08-16 14:54:06.181 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Proximal, control is Middle


In [13]:
method = INFERENCE_T_TEST
ap.run_de(method, PROTEINS, 'Distal', 'Proximal')
ap.run_de(method, PROTEINS, 'Distal', 'Middle')
ap.run_de(method, PROTEINS, 'Proximal', 'Middle')

ap.run_de(method, COMPOUNDS, 'Distal', 'Proximal')
ap.run_de(method, COMPOUNDS, 'Distal', 'Middle')
ap.run_de(method, COMPOUNDS, 'Proximal', 'Middle')

2021-08-16 14:54:20.977 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Distal, control is Proximal
2021-08-16 14:54:22.640 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Distal, control is Middle
2021-08-16 14:54:24.258 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Proximal, control is Middle
2021-08-16 14:54:25.827 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Distal, control is Proximal
2021-08-16 14:54:25.915 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Distal, control is Middle
2021-08-16 14:54:25.994 | INFO     | pyMultiOmics.pipelines:run_ttest:71 - t-test case is Proximal, control is Middle


## Query results

Retrieve a single node

In [14]:
node_id = '15366'
res = QueryBuilder(ap) \
        .add(Entity(node_id)) \
        .run()
res

,display_name,data_type,observed
entity_id,,,
15366,Acetic acid,compounds,False


Retrieve multiple nodes

In [15]:
node_id = ['15366', 'ENSDARG00000037781', 'F1QAA7']
res = QueryBuilder(ap) \
        .add(Entity(node_id)) \
        .run()
res

,display_name,data_type,observed
entity_id,,,
15366,Acetic acid,compounds,False
ENSDARG00000037781,Acss2,genes,True
F1QAA7,F1QAA7,proteins,True


Retrieve nodes connected to a single node

In [16]:
query_id = 'F1QAA7'
res = QueryBuilder(ap) \
        .add(Entity(query_id)) \
        .add(Connected()) \
        .run()
res

,display_name,data_type,observed,source_id
entity_id,,,,
ENSDARG00000037781,Acss2,genes,True,F1QAA7
15351,Acetyl-coa,compounds,False,F1QAA7
15366,Acetic acid,compounds,False,F1QAA7
16027,Adenosine 5-monophosphate,compounds,True,F1QAA7
30616,Atp(4-),compounds,False,F1QAA7
29888,Diphosphoric acid,compounds,False,F1QAA7
15346,Coenzyme a,compounds,False,F1QAA7
R-DRE-71735,Acetate + coa + atp => acetyl-coa + amp + pyro...,reactions,None,F1QAA7
R-DRE-71384,Ethanol oxidation,pathways,None,F1QAA7


Retrieve top-10 significantly changing genes

In [17]:
case = 'Distal'
control = 'Proximal'
pval = 0.05
fc_lte = -2
fc_gte = 2
N = 20

In [18]:
res = QueryBuilder(ap) \
        .add(Select(GENES)) \
        .add(SignificantDE(case, control, pval, fc_lte=fc_lte, fc_gte=fc_gte, N=N)) \
        .run()
res

,display_name,data_type,observed,padj_Distal_vs_Proximal,FC_Distal_vs_Proximal
ENSDARG00000045854,Fgf23,genes,True,3.569550e-10,24.000289
ENSDARG00000071091,Chrm3a,genes,True,3.307603e-11,22.486196
ENSDARG00000059090,Sstr2a,genes,True,2.760891e-09,21.559724
ENSDARG00000104259,Cabz01072885.1,genes,True,2.782933e-10,20.545646
ENSDARG00000005522,Galr1a,genes,True,2.091894e-09,20.531569
ENSDARG00000062758,Cr352265.1,genes,True,5.800455e-10,20.055404
ENSDARG00000093491,Si:dkey-78l4.8,genes,True,5.800455e-10,20.055404
ENSDARG00000076345,Taar19p,genes,True,6.804488e-11,20.002797
ENSDARG00000008541,Chia.4,genes,True,3.736906e-10,19.689389
ENSDARG00000037946,Prl,genes,True,3.736906e-10,19.689389


Find the compounds that are connected to the DE genes above

In [19]:
res = QueryBuilder(ap) \
        .add(Select(GENES)) \
        .add(SignificantDE(case, control, pval, fc_lte=fc_lte, fc_gte=fc_gte, N=N)) \
        .add(Connected(data_type=COMPOUNDS)) \
        .run()
res

,display_name,data_type,observed,source_id
entity_id,,,,
16474,Nadph,compounds,False,ENSDARG00000039067
18009,Nadp(+),compounds,False,ENSDARG00000039067
17347,Testosterone,compounds,False,ENSDARG00000039067
15378,Hydron,compounds,False,ENSDARG00000039067
16330,17beta-hydroxy-5alpha-androstan-3-one,compounds,False,ENSDARG00000039067
16990,Bilirubin ixalpha,compounds,False,ENSDARG00000103514


### Retrieve entity info

In [20]:
res = QueryBuilder(ap) \
        .add(Select(GENES)) \
        .add(SignificantDE(case, control, pval, fc_lte=fc_lte, fc_gte=fc_gte, N=N)) \
        .add(Connected()) \
        .add(Info()) \
        .run()
res

,display_name,data_type,observed,source_id,infos,images,links
entity_id,,,,,,,
Q1L8M3,Q1L8M3,proteins,False,ENSDARG00000045854,"[{'key': 'Name', 'value': 'Q1L8M3'}, {'key': '...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
X1WHZ7,X1WHZ7,proteins,False,ENSDARG00000071091,"[{'key': 'Name', 'value': 'Muscarinic acetylch...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
E7FDL0,E7FDL0,proteins,False,ENSDARG00000059090,"[{'key': 'Name', 'value': 'Somatostatin recept...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
A0A0G2L3I9,A0A0G2L3I9,proteins,False,ENSDARG00000104259,"[{'key': 'Name', 'value': 'A0A0G2L3I9'}, {'key...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
F1QSA8,F1QSA8,proteins,False,ENSDARG00000005522,"[{'key': 'Name', 'value': 'F1QSA8'}, {'key': '...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
F1QV82,F1QV82,proteins,False,ENSDARG00000062758,"[{'key': 'Name', 'value': 'Peptidase S1 domain...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
F1Q925,F1Q925,proteins,False,ENSDARG00000093491,"[{'key': 'Name', 'value': 'F1Q925'}, {'key': '...",[],"[{'text': 'Link to UniProt', 'href': 'http://w..."
F1QJL9,F1QJL9,proteins,False,ENSDARG00000076345,"[{'key': 'Name', 'value': 'F1QJL9'}, {'key': '...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."
F1RA25,F1RA25,proteins,False,ENSDARG00000008541,"[{'key': 'Name', 'value': 'Chitinase'}, {'key'...",[https://swissmodel.expasy.org/repository/unip...,"[{'text': 'Link to UniProt', 'href': 'http://w..."


In [21]:
from pyMultiOmics.info import get_info

In [22]:
entity_ids = ['ENSDARG00000091254', 'F1QAA7', '15378', 'R-DRE-469659', 'R-DRE-174403']
data_types = ['genes', 'proteins', 'compounds', 'reactions', 'pathways']
for entity_id, data_type in zip(entity_ids, data_types):
    print(entity_id, data_type)
    print(get_info(entity_id, data_type))
    print()

ENSDARG00000091254 genes
{'infos': [{'key': 'Description', 'value': 'si:ch73-59p9.2 '}, {'key': 'Species', 'value': 'danio_rerio'}], 'links': [{'text': 'Link to Ensembl', 'href': 'https://www.ensembl.org/id/ENSDARG00000091254'}, {'text': 'Link to GeneCard', 'href': 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=si:ch73-59p9.2'}, {'text': 'Transcript: si:ch73-59p9.2-201', 'href': 'https://www.ensembl.org/id/ENSDART00000111526'}], 'images': []}

F1QAA7 proteins
{'infos': [{'key': 'Name', 'value': 'Propionate--CoA ligase'}, {'key': 'EC Number', 'value': 'EC6.2.1.17'}, {'key': 'Catalytic Activity', 'value': '\n\nATP + CoA + propanoate = AMP + diphosphate + propanoyl-CoA\n\n\n\n\n\n\n\n\n\n\n\n\n'}, {'key': 'Catalytic Activity', 'value': '\n\nacetate + ATP + CoA = acetyl-CoA + AMP + diphosphate\n\n\n\n\n\n\n\n\n\n\n\n\n'}, {'key': 'Gene_ontologies', 'value': 'acetate-CoA ligase activity; propionate-CoA ligase activity'}], 'links': [{'text': 'Link to UniProt', 'href': 'http://www.unipro

### Plot some heatmap using Plotly

In [23]:
res = QueryBuilder(ap) \
        .add(Select(GENES)) \
        .add(SignificantDE(case, control, pval, fc_lte=fc_lte, fc_gte=fc_gte, N=N)) \
        .run()
res

,display_name,data_type,observed,padj_Distal_vs_Proximal,FC_Distal_vs_Proximal
ENSDARG00000045854,Fgf23,genes,True,3.569550e-10,24.000289
ENSDARG00000071091,Chrm3a,genes,True,3.307603e-11,22.486196
ENSDARG00000059090,Sstr2a,genes,True,2.760891e-09,21.559724
ENSDARG00000104259,Cabz01072885.1,genes,True,2.782933e-10,20.545646
ENSDARG00000005522,Galr1a,genes,True,2.091894e-09,20.531569
ENSDARG00000062758,Cr352265.1,genes,True,5.800455e-10,20.055404
ENSDARG00000093491,Si:dkey-78l4.8,genes,True,5.800455e-10,20.055404
ENSDARG00000076345,Taar19p,genes,True,6.804488e-11,20.002797
ENSDARG00000008541,Chia.4,genes,True,3.736906e-10,19.689389
ENSDARG00000037946,Prl,genes,True,3.736906e-10,19.689389


In [24]:
data_type = GENES
analysis = ap.get_de_analysis(data_type, case, control)
wi = analysis.wi
data_df, design_df = wi.data_df, wi.design_df
data_df

,US-1584693,US-1584700,US-1584706,US-1584712,US-1584722,US-1584724,US-1584725,US-1584732,US-1584738,US-1584744,US-1584752,US-1584753,US-1584754,US-1584758,US-1584765
Identifier,,,,,,,,,,,,,,,
ENSDARG00000000001,51.0,40.0,69.0,78.0,89.0,47.0,88.0,86.0,110.0,55.0,54.0,58.0,104.0,43.0,72.0
ENSDARG00000000002,283.0,129.0,164.0,269.0,211.0,171.0,146.0,256.0,283.0,150.0,223.0,142.0,272.0,260.0,256.0
ENSDARG00000000018,545.0,503.0,547.0,387.0,332.0,559.0,623.0,499.0,436.0,488.0,549.0,462.0,287.0,495.0,299.0
ENSDARG00000000019,437.0,469.0,538.0,557.0,550.0,404.0,544.0,443.0,623.0,502.0,487.0,470.0,460.0,329.0,480.0
ENSDARG00000000068,266.0,249.0,247.0,236.0,195.0,247.0,283.0,259.0,299.0,232.0,249.0,231.0,236.0,274.0,241.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSDARG00000105304,1.0,0.6,2.0,1.0,0.4,0.4,1.0,0.4,1.0,0.6,1.0,0.6,0.4,0.4,0.4
ENSDARG00000105305,14.0,20.0,15.0,21.0,21.0,20.0,30.0,25.0,34.0,27.0,16.0,20.0,24.0,24.0,18.0
ENSDARG00000105306,6.0,1.0,1.0,2.0,2.0,3.0,1.0,3.0,1.0,6.0,3.0,1.0,4.0,1.0,3.0


In [25]:
case_group = design_df[design_df['group'] == case].index.values.tolist()
control_group = design_df[design_df['group'] == control].index.values.tolist()
selection = case_group + control_group
selection

['US-1584738',
 'US-1584722',
 'US-1584712',
 'US-1584754',
 'US-1584765',
 'US-1584752',
 'US-1584732',
 'US-1584724',
 'US-1584693',
 'US-1584758']

In [26]:
heatmap_df = wi.data_df.loc[res.index.values]
heatmap_df = heatmap_df[selection]
heatmap_df

,US-1584738,US-1584722,US-1584712,US-1584754,US-1584765,US-1584752,US-1584732,US-1584724,US-1584693,US-1584758
Identifier,,,,,,,,,,
ENSDARG00000045854,28.000000,22.000000,27.000000,8.000000,10.000000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000071091,5.000000,4.000000,7.000000,5.000000,10.000000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000059090,2.800000,9.000000,4.000000,2.800000,1.000000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000104259,3.000000,1.400000,1.000000,2.000000,1.000000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000005522,5.000000,1.000000,1.000000,1.600000,1.000000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000062758,3.000000,1.000000,0.800000,0.800000,0.800000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000093491,0.800000,0.800000,0.800000,3.000000,1.000000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000076345,1.000000,1.000000,2.000000,0.800000,0.800000,0.000001,0.000001,0.000001,0.000001,0.000001
ENSDARG00000008541,0.600000,1.000000,0.600000,2.000000,0.600000,0.000001,0.000001,0.000001,0.000001,0.000001


In [28]:
from plotly import express as px
px.imshow(wi.standardize_df(heatmap_df, log=True, axis=0))